In [2]:
#LIBRERIAS
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.signal import find_peaks

In [3]:
# Ruta del archivo Excel
ruta_archivo = '/content/consolidado_motores.xlsx'
# Cargar el archivo Excel en un DataFrame
consolidado = pd.read_excel(ruta_archivo, sheet_name='Consolidado')

# ANALISIS GENERAL

In [ ]:
# Seleccionar solo las columnas de RPM y Potencia
rpm_potencia = consolidado[['R.P.M', 'POTENCIA (HP)']]

# Calcular estadísticas descriptivas para RPM y Potencia
estadisticas_rpm_potencia = rpm_potencia.describe()

# Imprimir la tabla resumen
print("\nTabla Resumen de RPM y Potencia:")
estadisticas_rpm_potencia

In [ ]:
# Calcular la cantidad de datos nulos por columna
cantidad_nulos = consolidado.isnull().sum()

# Imprimir la cantidad de datos nulos por columna
print("\nCantidad de datos nulos por columna:")
cantidad_nulos

In [ ]:
potencia = consolidado['POTENCIA (HP)']
conteos = potencia.value_counts()
print(conteos)

In [ ]:
voltajes = consolidado['VOLTAJE']
conteos = voltajes.value_counts()
print(conteos)

# SEGMENTACIÓN DE VOLTAJE

CANTIDAD DE MOTORES EN CADA VOLTAJE

In [ ]:
# Contar el número de registros en cada categoría
conteos = consolidado['SEG_VOLT'].value_counts()
print(conteos)

In [ ]:
# Contar el número de registros en cada categoría
conteos = consolidado['SEG_VOLT'].value_counts()

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))  # Ajustar el tamaño del gráfico
plt.bar(conteos.index, conteos.values)
plt.xlabel("Segmento de Voltaje")
plt.ylabel("Cantidad de Motores")
plt.title("Cantidad de Motores por Segmento de Voltaje")

# Agregar etiquetas con las cantidades
for i, v in enumerate(conteos.values):
    plt.text(i, v + 0.5, str(v), ha='center', va='bottom')

plt.xticks(rotation=45, ha='right')  # Rotar las etiquetas del eje x para mejor visualización
plt.tight_layout()  # Ajustar el diseño para evitar superposición
plt.show()


CANTIDAD DE MOTORES POR PROCESO SEGMENTADO POR VOLTAJE

In [ ]:
conteo_seg_volt_por_proceso = consolidado.groupby(['PROCESO', 'SEG_VOLT']).size()
print(conteo_seg_volt_por_proceso)


GRAFICO (Se puede mejorar)

In [ ]:
# Definir una paleta de colores agradable usando seaborn
paleta_colores = sns.color_palette("husl", 5)  # Puedes cambiar "husl" por otras paletas como "coolwarm", "Set2", etc.

# Agrupar los datos por proceso y seg_volt
conteo_seg_volt_por_proceso = consolidado.groupby(['PROCESO', 'SEG_VOLT']).size()

# Crear el histograma con la nueva paleta de colores
ax = conteo_seg_volt_por_proceso.unstack().plot(kind='bar', figsize=(15, 8), color=paleta_colores)

# Agregar el valor de cada segmento en las barras
for p in ax.patches:
    height = p.get_height()
    if height > 0:  # Para evitar etiquetas de valor en barras de altura 0
        x = p.get_x() + p.get_width() / 2
        y = height
        ax.text(x, y, str(int(height)), ha='center', va='bottom')

plt.xlabel('Proceso')
plt.ylabel('Cantidad')
plt.title('Cantidad por Proceso, segmentado por SEG_VOLT')
plt.xticks(rotation=45, ha='right')
plt.legend(title='SEG_VOLT')
plt.tight_layout()
plt.show()


# ANALISIS DE RPM

CONTEO DE MOTORES POR RPM

In [ ]:
conteos = consolidado['R.P.M'].value_counts()
print(conteos)

In [ ]:
conteos = consolidado['Frecuencia'].value_counts()
print(conteos)

In [ ]:
conteos = consolidado['Polos'].value_counts()
print(conteos)

**Distribución R.P.M**

In [ ]:
# Crear el histograma de la columna 'R.P.M'
conteos, bins, patches = plt.hist(consolidado['R.P.M'], bins=10)  # Ajusta el número de bins según sea necesario

# Agregar etiquetas en la parte superior de cada barra
for count, x in zip(conteos, bins):
    plt.text(x + (bins[1] - bins[0]) / 2,  # Posición horizontal, centrada en cada barra
             count,  # Posición vertical (altura del texto)
             str(int(count)),  # Etiqueta de texto
             ha='center', va='bottom', fontsize=10)  # Alineación y tamaño de fuente

# Crear etiquetas personalizadas para los ticks del eje X utilizando los rangos
bin_labels = [f'{int(bins[i])}-{int(bins[i+1])}' for i in range(len(bins)-1)]
plt.xticks(ticks=bins[:-1] + (bins[1] - bins[0]) / 2, labels=bin_labels, rotation=45)
# Configurar el color de fondo del gráfico

# Configurar etiquetas y título del gráfico
plt.xlabel('R.P.M')
plt.ylabel('Frecuencia')
plt.title('Distribución de R.P.M')
# Mostrar el gráfico
plt.show()


# ANALISIS DE POTENCIA

In [ ]:
# Agrupar por potencia y contar las ocurrencias
conteos_potencia = consolidado['POTENCIA (HP)'].value_counts().sort_index()

# Crear el gráfico de línea
plt.plot(conteos_potencia.index, conteos_potencia.values)
plt.xlabel('Potencia (HP)')
plt.ylabel('Recuento')
plt.title('Distribución de Potencia(0.25 - 400 HP)')
plt.xlim(min(consolidado['POTENCIA (HP)']), 400)  # Establecer el rango del eje x de 0 a 400
plt.show()


In [ ]:
# Crear el gráfico de línea
plt.plot(conteos_potencia.index, conteos_potencia.values)
plt.xlabel('Potencia (HP)')
plt.ylabel('Recuento')
plt.title('Distribución de Potencia')
plt.xlim(400, max(consolidado['POTENCIA (HP)']))
plt.show()

In [ ]:
# Agrupar por potencia y contar las ocurrencias
conteos_potencia = consolidado['POTENCIA (HP)'].value_counts().sort_index()

# Filtrar datos para el rango de 0 a 250
conteos_potencia_filtrado = conteos_potencia[(conteos_potencia.index >= 0) & (conteos_potencia.index <= 250)]

# Encontrar los picos locales
peaks, _ = find_peaks(conteos_potencia_filtrado.values, height=20)  # Ajusta el valor de 'height' según sea necesario

# Encontrar el pico global
max_value = conteos_potencia_filtrado.max()
max_index = conteos_potencia_filtrado.idxmax()

# Crear el gráfico de línea
plt.plot(conteos_potencia_filtrado.index, conteos_potencia_filtrado.values)
plt.xlabel('Potencia (HP)')
plt.ylabel('Recuento')
plt.title('Distribución de Potencia (0.25 - 250 HP)')
plt.xlim(0, 250)  # Establecer el rango del eje x de 0 a 250

# Etiquetar los picos locales
for peak in peaks:
  plt.plot(conteos_potencia_filtrado.index[peak], conteos_potencia_filtrado.values[peak], 'ro')
  plt.text(conteos_potencia_filtrado.index[peak], conteos_potencia_filtrado.values[peak],
            f'{conteos_potencia_filtrado.values[peak]}', ha='center', va='bottom')

# Etiquetar el pico global
plt.plot(max_index, max_value, 'go')
plt.text(max_index, max_value, f'{max_value}', ha='center', va='bottom')

plt.show()


In [ ]:
# Filtrar el DataFrame para incluir solo potencias entre 0.25 y 250
filtered_df = consolidado[(consolidado['POTENCIA (HP)'] >= 0.25) & (consolidado['POTENCIA (HP)'] <= 250)]

# Crear el histograma
n, bins, patches = plt.hist(filtered_df['POTENCIA (HP)'], bins=10)  # Ajusta el número de bins según sea necesario

# Agregar etiquetas en la parte superior de cada barra
for count, x in zip(n, bins):
    plt.text(x + (bins[1] - bins[0]) / 2,  # Posición horizontal, centrada en cada barra
             count,  # Posición vertical (altura del texto)
             str(int(count)),  # Etiqueta de texto
             ha='center', va='bottom', fontsize=10)  # Alineación y tamaño de fuente

# Crear etiquetas personalizadas para los ticks del eje X utilizando los rangos
bin_labels = [f'{int(bins[i])}-{int(bins[i+1])}' for i in range(len(bins)-1)]
plt.xticks(ticks=bins[:-1] + (bins[1] - bins[0]) / 2, labels=bin_labels, rotation=45)

# Configurar etiquetas y título del gráfico
plt.xlabel('Potencia (HP)')
plt.ylabel('Frecuencia')
plt.title('Distribución de Potencia (0.25-250 HP)')
plt.show()
